In [1]:
import ginza # pip install -U ginza ja_ginza
import nltk  # pip install nltk
import spacy # pip install nltk
import pke   # pip install git+https://github.com/boudinfl/pke.git
import pandas as pd

In [2]:
df = pd.read_csv('datasets/bachelor_themes.csv')
df.head()

,年度,学科／研究科,研究室,学籍番号,賞,氏名,要旨,題目
0,H17,情報科学科,相研,NaN,NaN,NaN,NaN,WEBページにおけるユーザビリティの追及
1,H17,情報科学科,松永研,NaN,NaN,NaN,NaN,安全運転管理教育システム(ASSIST)における複数カメラ使用での交通事故防止対策
2,H17,情報科学科,宮崎研,NaN,NaN,NaN,NaN,電子透かしの体制に関する検討 -加重平均フィルタ・メディアンフィルタの場合-
3,H17,情報科学科,成研,NaN,NaN,NaN,NaN,対面教育を支援するためのウェブ助言システム
4,H17,情報科学科,石田健研,NaN,NaN,NaN,NaN,衛星通信検討のための気象データの解析-2002年のデータ解析-


In [5]:
target_labs = [
    '朝廣研','安部研','石田健研','合志研','下川研',
    '成研','田中研','仲研','米元研','アプドゥハン研',
    '稲永研','于研','石田俊研','澤田研','安武研','前田研']

target_df = df[df["研究室"].isin(target_labs )] 
target_df.head() 

,年度,学科／研究科,研究室,学籍番号,賞,氏名,要旨,題目
3,H17,情報科学科,成研,NaN,NaN,NaN,NaN,対面教育を支援するためのウェブ助言システム
4,H17,情報科学科,石田健研,NaN,NaN,NaN,NaN,衛星通信検討のための気象データの解析-2002年のデータ解析-
6,H17,情報科学科,仲研,NaN,NaN,NaN,NaN,生化学反応シミュレータの比較評価
7,H17,情報科学科,澤田研,NaN,NaN,NaN,NaN,FPGAを搭載したロボット教材のためのIPとしてKUE-CHIP2の検討
9,H17,情報科学科,石田健研,NaN,NaN,NaN,NaN,重畳積文法によるX線CTの画像再構成-Shepp-Loganのフィルタ関数の適用-


In [3]:
years = df["年度"].unique()
years

array(['H17', 'H18', 'H19', 'H20', 'H21', 'H22', 'H23', 'H24', 'H25',
       'H26', 'H27', 'H28', 'H29', 'H30', 'R01', 'R02', 'R03', 'R04'],
      dtype=object)

In [19]:
docs = {}
for y in years:
    for b in target_labs:
        titles = target_df[(target_df['年度']==y) & (target_df['研究室']==b)]['題目'].values.tolist()
        # print(f"{y}:{b}")
        docs[(y, b)] = titles

r04_ishida = docs['R04', '石田健研']
print('。'.join(r04_ishida))

MathematicaとMathTeXを用いた線形代数の教材作成 ―4次の行列式、クラーメルの公式―。MathematicaとMathTeXを用いた微分積分の教材作成 ―合成関数の微分と有理数べき関数の追加―。プログラミング環境による連立一次方程式求解の速度比較。最小点の探索におけるアルゴリズムの調査 ―探索アルゴリズムによる実行時間の比較―。層状媒質の1次元FDTD法解析 ―パルスの反射波から層状媒質の比誘電率分布の推定―。マイクロ波を用いる誘電体円柱の誘電率推定に対するMATLABニューラルネットワークの検証。九産大12号館建物内に侵入したローカル5Gの電波測定


In [20]:
pke.lang.stopwords['ja_ginza'] = 'ja'
def get_key_phrase(spacy_model, text, n = 5, stopwords=None):
    """キーフレーズ処理

    Args:
        text ([type]): キーフレーズ対象の文字列
        n (int, optional): [description]. Defaults to 10.
    Returns:
        [type]: [description]
    """
    #MultipartiteRankなどのキーフレーズの手法については以下のサイトが参考になります
    #https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part5.html
    extractor = pke.unsupervised.MultipartiteRank()
    extractor.load_document(input=text, language='ja', normalization=None, stoplist=stopwords)
    extractor.candidate_selection( pos={'NOUN', 'PROPN', 'ADJ', 'NUM'})
    extractor.candidate_weighting(threshold=0.10, method='average', alpha=1.1)
    key_phrase = extractor.get_n_best(n)
    
    return  key_phrase

In [21]:
from spacy.lang.ja import stop_words
spacy_model = spacy.load("ja_ginza")  # GiNZAモデルの読み込み
stopwords = list(stop_words.STOP_WORDS)
print(stopwords[-10:])
nltk.corpus.stopwords.words = stopwords

['しまう', 'くる', 'もっ', 'れ', 'ため', 'られ', 'ず', 'まま', 'すぐ', 'かつて']


In [22]:
text = ". ".join(r04_ishida)
get_key_phrase(spacy_model, text, n=100, stopwords=stopwords)

[('教材 作成', 0.09755936806166772),
 ('層状 媒質', 0.06405934272182079),
 ('mathematica', 0.060735485513285406),
 ('誘電体 円柱', 0.05455976109737337),
 ('誘電率 推定', 0.054461313725697715),
 ('合成 関数', 0.05308295191254813),
 ('微分積分', 0.05079275303558475),
 ('クラーメル', 0.04943679193116122),
 ('行列式', 0.049062031181767427),
 ('matlab ニューラル ネットワーク', 0.045936827657894086),
 ('探索 アルゴリズム', 0.04592338929339313),
 ('実行 時間', 0.044607783203554234),
 ('線形 代数', 0.04328374574140909),
 ('マイクロ波', 0.04289309401329305),
 ('有理数', 0.04104918966911821),
 ('アルゴリズム', 0.038986663836444904),
 ('パルス', 0.03629098534918045),
 ('九産大 12 号館 建物内', 0.03557986937903461),
 ('プログラミング 環境', 0.03490146023960468),
 ('速度 比較', 0.033247537629381395),
 ('電波 測定', 0.02354965480678551)]

In [44]:
import itertools
from pprint import pprint

for b, y in itertools.product(target_labs[:5], years[-4:]):
    text = ". ".join(docs[y, b])
    keyphrases = get_key_phrase(spacy_model, text, n=100, stopwords=stopwords)
    print(f"{b}-{y}")
    pprint(keyphrases)


朝廣研-R01
[('java 言語', 0.12773132102163692),
 ('アルゴリズム', 0.11439320212429407),
 ('性能 評価', 0.0788417226103075),
 ('グラフ 理論', 0.07830191515351204),
 ('演習 教材', 0.07588031729052523),
 ('プログラミング ロボット', 0.06995218753790589),
 ('展示物', 0.067986257724208),
 ('maxima', 0.06794886831862666),
 ('自然数', 0.06593948038902424),
 ('プログラム 教材', 0.06068034161097681),
 ('アルゴ', 0.059252197327079585),
 ('ウェブブラウザ', 0.05554120675286211),
 ('オンライン テスト', 0.040879894630888466),
 ('sram モデル', 0.03667108750815271)]
朝廣研-R02
[('musicxml', 0.10803719386944957),
 ('性能 評価', 0.10689368805919766),
 ('アルゴリズム', 0.0996078124392605),
 ('地図 データ 試作', 0.09757697614201795),
 ('opensheetmusicdisplay', 0.09024256369815613),
 ('地図 データ', 0.07720192968188332),
 ('導関数', 0.07573046112585173),
 ('ツール', 0.07378177176199892),
 ('svg ファイル', 0.07377825143035098),
 ('自然数', 0.07071962222256852),
 ('実験的', 0.06336352449523379),
 ('プログラム', 0.06306620507403068)]
朝廣研-R03
[('ai 開発', 0.1073940079954291),
 ('マンカラ', 0.10681531304301678),
 ('性能 評価', 0.07503